In [6]:
# 1. 生成示例数据
import numpy as np
import pandas as pd

# 生成示例数据
np.random.seed(0)
X = np.random.randn(1000, 10)  # 1000个样本，每个样本有10个特征
y = np.random.randint(0, 2, size=1000)  # 二分类标签，0或1

# 将数据保存到DataFrame中
data = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(1, 11)])
data['target'] = y

# 保存数据到CSV文件
data.to_csv('example_data.csv', index=False)

In [7]:
# 2. 加载数据并预处理
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 加载数据
df1 = pd.read_csv('RESULT.oa.csv')
df2 = pd.read_csv('RESULT.non oa.csv')
data = pd.concat([df1, df2], axis=0)  # axis=0表示垂直堆叠，axis=1表示水平堆叠
print(data.shape)
# 分割特征和标签
X = data.drop('label', axis=1).values
y = data['label'].values

# 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 分割训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(X_train)


(709, 131)
[[ 1.08390342  1.38498757 -0.75780429 ...  0.70659736 -0.10815092
   0.84011998]
 [-0.92259142 -0.70521717 -0.72745559 ... -0.14768158 -0.18485242
  -0.4515663 ]
 [-0.92259142 -0.70521717  0.94931    ... -0.1037773   0.9584246
  -0.56689339]
 ...
 [-0.92259142 -0.15516329  1.83195131 ... -0.86260966  0.12100491
  -0.84171389]
 [-0.92259142 -1.58530338 -0.32280628 ...  0.62418499 -0.38311137
  -0.10307029]
 [ 1.08390342 -0.5952064   0.34739414 ... 11.67144906  0.93634662
   7.65554602]]


In [8]:
# 3. 使用PCA进行降维
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
# 使用PCA
pca = PCA()# 使用PCA进行降维
pca = PCA(n_components=0.95, random_state=42)  # 选择保留95%方差的主成分
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# 输出降维后的特征数
print(f"降维后的特征数: {X_train_pca.shape[1]}")  # 输出降维后的维度数

降维后的特征数: 31


In [9]:
# 4. 构建和训练神经网络模型
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam  # 确保正确导入
# 定义神经网络模型
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_pca.shape[1],)))  # 更新输入层形状为降维后的特征数
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # 二分类任务，使用sigmoid作为输出层激活函数

# 编译模型
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# 训练模型
history = model.fit(X_train_pca, y_train, epochs=100, batch_size=32, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate(X_test_pca, y_test)
print('Test accuracy:', test_acc)

# 保存模型
model.save('my_model.keras')


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                2048      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4161 (16.25 KB)
Trainable params: 4161 (16.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/100
15/15 [==============================] - 4s 40ms/step - loss: 0.7604 - accuracy: 0.5386 - val_loss: 0.7317 - val_accuracy: 0.6053
Epoch 2/100
15/15 [==============================] - 0s 10ms/step - loss: 0.7395 - accuracy: 0.5651 - val_loss: 0.7157 - val_accuracy: 0.6228
Epoch 3/100
15/15 [==============================] - 0s 10ms/step - loss: 0.7194 - accuracy: 0.5784 - val_loss: 0.7043 - val_accuracy: 0.6404
Epoch 4/100
15/15 [==============================] - 0s 9ms/step - loss: 0.7036 - accuracy: 0.5740 - val_loss: 0.6934 - val_accuracy: 0.6404
Epoch 5/100
15/15 [==============================] - 0s 9ms/step - loss: 0.6893 - accuracy: 0.5938 - val_loss: 0.6845 - val_accuracy: 0.6404
Epoch 6/100
15/15 [==============================] - 0s 9ms/step - loss: 0.6773 - accuracy: 0.6026 - val_loss: 0.6757 - val_accuracy: 0.6228
Epoch 7/100
15/15 [==============================] - 0s 9ms/step - loss: 0.6653 - accuracy: 0.6093 - val_loss: 0.6697 - val_accuracy: 0.6316
Epoch 8/10

In [10]:
#  加载 Keras 本地格式的模型
model = tf.keras.models.load_model('my_model.keras')

# 生成一些新的示例数据（这里假设有新的数据）
X_new = pd.read_csv("RESULT.oa.csv")  # 5个样本，每个样本有130个特征

X = X_new.drop('label', axis=1).values

# 数据标准化（使用之前的scaler）
X_new_scaled = scaler.transform(X)

# PCA降维（使用之前的PCA）
X_new_pca = pca.transform(X_new_scaled)

# 使用模型进行预测
predictions = model.predict(X_new_pca)

# 输出预测结果
print(predictions)

11/11 [==============================] - 0s 3ms/step
[[0.6083752 ]
 [0.0683864 ]
 [0.8943182 ]
 [0.8134248 ]
 [0.993692  ]
 [0.67131513]
 [0.89567095]
 [0.31867954]
 [0.8799629 ]
 [0.8930397 ]
 [0.9967328 ]
 [0.59705687]
 [0.3455385 ]
 [0.9832154 ]
 [0.8611443 ]
 [0.93167776]
 [0.91859156]
 [0.6368263 ]
 [0.952737  ]
 [0.5157976 ]
 [0.9515654 ]
 [0.46502864]
 [0.9759738 ]
 [0.9585403 ]
 [0.9694546 ]
 [0.96262956]
 [0.7463692 ]
 [0.4618582 ]
 [0.89524907]
 [0.9122158 ]
 [0.33624956]
 [0.6857974 ]
 [0.91090816]
 [0.46149263]
 [0.814458  ]
 [0.5611928 ]
 [0.7805537 ]
 [0.98335695]
 [0.99581105]
 [0.5429908 ]
 [0.5614386 ]
 [0.34241372]
 [0.2981437 ]
 [0.76012754]
 [0.27912122]
 [0.8865684 ]
 [0.8695651 ]
 [0.42888278]
 [0.90668243]
 [0.90119034]
 [0.9564345 ]
 [0.964107  ]
 [0.8425451 ]
 [0.9196303 ]
 [0.17885415]
 [0.67137855]
 [0.34680757]
 [0.6546323 ]
 [0.60506797]
 [0.9201752 ]
 [0.8845589 ]
 [0.81830984]
 [0.9613549 ]
 [0.68003625]
 [0.42965266]
 [0.81015325]
 [0.66020024]
 [0.78654